In [1]:
# install.packages('languageserver', dependencies = TRUE)
# install.packages('devtools', dependencies = TRUE)
# devtools::install_github('IRkernel/IRkernel',dependencies = TRUE)
# IRkernel::installspec()
# devtools::install_github("mrcieu/gwasglue")
# install_github("jrs95/gassocplot",dependencies=TRUE,upgrade="always")

In [10]:
suppressPackageStartupMessages(suppressWarnings({
    library(gwasglue)
    library(dplyr)
    library(gassocplot)
    library(coloc)
    library(ggplot2)
    library(foreach)
    library(doParallel)
}))
# ncpus <- detectCores()
registerDoParallel(1)


In [3]:
# wbc count related to autoimmune disease
gwas_dataset = "ieu-b-30"
dummy_dataset = "ieu-a-7"
gwasinfo(id=as.character(gwas_dataset))
gwasinfo(id=as.character(dummy_dataset))

id,trait,category,unit,population,priority,author,year,mr,sex,consortium,sample_size,group_name,ontology,subcategory,build,note
<chr>,<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<lgl>,<chr>,<chr>,<lgl>
ieu-b-30,white blood cell count,Continuous,NA,European,0,"Vuckovic, D",2020,1,Males and Females,Blood Cell Consortium,563946,public,NA,Immune system,HG19/GRCh37,NA


id,trait,pmid,category,unit,ncontrol,population,priority,author,year,ncase,nsnp,mr,sex,consortium,sample_size,group_name,ontology,subcategory,build
<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<lgl>,<chr>,<chr>
ieu-a-7,Coronary heart disease,26343387,Disease,log odds,123504,Mixed,1,Nikpay,2015,60801,9455779,1,Males and Females,CARDIoGRAMplusC4D,184305,public,NA,Cardiovascular,HG19/GRCh37


In [4]:
top <- ieugwasr::tophits('ieu-b-30') %>% arrange(p)
str(top)

tibble [509 × 12] (S3: tbl_df/tbl/data.frame)
 $ chr     : chr [1:509] "6" "17" "12" "4" ...
 $ position: int [1:509] 31236115 38089344 111884608 74948054 28724374 44153100 32209861 25218243 45744842 36943916 ...
 $ beta    : num [1:509] 0.0772 0.0721 -0.0672 -0.0632 0.0593 ...
 $ se      : num [1:509] 0.00188 0.00186 0.00184 0.0019 0.00204 ...
 $ p       : num [1:509] 0.00 0.00 3.69e-293 3.50e-243 2.73e-185 ...
 $ n       : num [1:509] 561159 561295 559426 562241 561153 ...
 $ id      : chr [1:509] "ieu-b-30" "ieu-b-30" "ieu-b-30" "ieu-b-30" ...
 $ rsid    : chr [1:509] "rs2853951" "rs4794820" "rs3184504" "rs1371794" ...
 $ ea      : chr [1:509] "T" "G" "C" "C" ...
 $ nea     : chr [1:509] "C" "A" "T" "A" ...
 $ eaf     : num [1:509] 0.6 0.552 0.517 0.622 0.295 ...
 $ trait   : chr [1:509] "white blood cell count" "white blood cell count" "white blood cell count" "white blood cell count" ...


In [11]:
window_size <- 50000
if (!dir.exists("gwas_hits")) {
    dir.create("gwas_hits")
}
# loop over each chr in parallel? This is almost twice as fast as plain for loop but
# not sure why since only one core is being used (registering >1 core throws errors)
foreach (i=1:22) %dopar% {

    count <- 0
    top_hit_pos <- top[top$chr == i, ]$position
    # loop over each position in top hit positions and calculate window to retrieve all SNPs from
    for (j in top_hit_pos) {
        lower <- j - window_size
        upper <- j + window_size
        chrpos <- paste0(i, ":", lower, "-", upper)

        # extract SNPS in the region of top hit
        out <- ieugwasr_to_coloc(
            id1 = as.character(gwas_dataset),
            id2 = as.character(dummy_dataset),
            chrompos = as.character(chrpos),
            type1 = "quant",
            # type2 = "cc"
        )
        
        out <- out[1]
        # str(out)

        # export to file (tab-delimited) in directory named gwas_hits, don't wrap in quotes, no headers
        write.table(out,
            file = paste0(
                "gwas_hits/", i, "_", lower, "_",upper, "_gwas.tsv"
            ),
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE
        )
        # increment count by 1
        # count <- count + 1

        # print progress for each chromosome
        # print(paste0(
        #     count, " of ",
        #     length(top_hit_pos),
        #     " top hits for chr ", i," processed"
        # ))
    }
}

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.

Type information not available for ieu-b-30. Assuming 'quant' but override using 'type' arguments.



[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

In [ ]:
# python3 eqtl_build_db.py
# python3 eqtl_query_db.py

In [ ]:
# get list of files in output_fdr directory
files <- list.files(path = "output", pattern = ".*.tsv", full.names = TRUE)

# truncate filelist to first pair of files
# files <- files[1:4]
# files

# This may need to be modified so it's not just taking file names in alphabetical order, but in descending pval order.
for (i in seq(1, length(files), 2)) {
    print(paste("starting",i,files[i],files[i+1]))

    # read in files
    eqtl <- as.list(read.table(file=files[i], header = TRUE, sep = '\t'))
    gwas <- as.list(read.table(file=files[i+1], header = TRUE, sep = '\t'))
    
    # run coloc and write the output to a log file
    res <- coloc::coloc.abf(gwas, eqtl) 
    
    # # make a directory named output_coloc if it doesn't exist
    dir.exists("output_coloc") || dir.create("output_coloc")

    # print numeric value of element of res$summary named PP.H4.abf and filename without extension and without _eQTLS
    # str(res)
    # print(as.numeric(res$summary["PP.H4.abf"]))
    #  remove file extension from files[i] and store in a variable
    SNP_locus_name <- basename(files[i])
    SNP_locus_name <- sub("_eQTLs\\..*$", "", SNP_locus_name)
    SNP_locus_name <- sub("^region_", "", SNP_locus_name)
    
    # write output to a log file, tab-separated
    write.table(paste(SNP_locus_name,as.numeric(res$summary["PP.H1.abf"]),as.numeric(res$summary["PP.H2.abf"]),as.numeric(res$summary["PP.H3.abf"]),as.numeric(res$summary["PP.H4.abf"])), file = paste0("output_coloc/", "PP.H1-4.abf.tsv"), sep = "\t", quote=FALSE, row.names = FALSE, col.names = FALSE, append = TRUE)
    

    # print(paste("eQTL path: ", files[i]))
    # print(paste("GWAS path: ", files[i+1]))
    # create a new list where eqtl and gwas are its elements
    out <- list(eqtl, gwas)

    print(paste("ending",i,files[i],files[i+1]))
}

In [ ]:
# asr::tophits('ieu-a-300') %>% arrange(p)   
# top 
# chrpos <- paste0(top$chr[1], ":", top$position[1] - 90000, "-", top$position[1] + 90000)
# out <- ieugwasr_to_coloc(id1='ieu-a-300', id2='ieu-a-7', chrompos=chrpos)
# res <- coloc::coloc.abf(out[[1]], out[[2]])
# temp <- coloc_to_gassocplot(out)
# gassocplot::stack_assoc_plot(temp$markers, temp$z, temp$corr, traits=temp$traits)

# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-300/ieu-a-300.vcf.gz","ieu-a-300.vcf.gz")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-300/ieu-a-300.vcf.gz.tbi","ieu-a-300.vcf.gz.tbi")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-7/ieu-a-7.vcf.gz","ieu-a-7.vcf.gz")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-7/ieu-a-7.vcf.gz.tbi","ieu-a-7.vcf.gz.tbi")

# chrpos <- "19:11112306-11292306"
# vout <- gwasvcf_to_coloc("ieu-a-300.vcf.gz", "ieu-a-7.vcf.gz", chrpos)
# vres <- coloc::coloc.abf(vout[[1]], vout[[2]])

# library(gassocplot)
# temp <- coloc_to_gassocplot(vout)
# gassocplot::stack_assoc_plot(temp$markers, temp$z, temp$corr, traits=temp$traits)